In [ ]:
import tweepy
from tqdm.auto import tqdm
from IPython.display import display, HTML
import re

## tweepy client

tweepy v4
https://docs.tweepy.org/en/stable/client.html

In [ ]:
API_KEY = os.getenv('TWITTER_BOT_KEY')
API_SECRET_KEY = os.getenv('TWITTER_BOT_SECRET_KEY')

ACCESS_TOKEN = os.getenv('rbtt_ACCESS_TOKEN')
ACCESS_TOKEN_SECRET = os.getenv('rbtt_ACCESS_TOKEN_SECRET')


client = tweepy.Client(
    consumer_key=API_KEY,
    consumer_secret=API_SECRET_KEY,
    access_token=ACCESS_TOKEN,
    access_token_secret=ACCESS_TOKEN_SECRET,
    wait_on_rate_limit=True
)
client.get_me()

In [ ]:
target = client.get_user(username='', user_fields=['public_metrics', 'description', 'profile_image_url'], user_auth=True).data
display(HTML(f'''
    <img src={target.profile_image_url}>
    {target.description}<br>
    {target.public_metrics}
'''))

`max_results`: results per page

`limit`: max number of pages

`flatten()` to loop as if there's no `Paginator`

#### Followers insight using `.get_users_followers()`

1 <= `max_results` <= 1000

In [ ]:
mr = 1000
lim = target.public_metrics['followers_count'] // mr + 1
sorted_flwrs = []
for user in tqdm(tweepy.Paginator(client.get_users_followers,
                    target.id, user_fields=['public_metrics', 'profile_image_url'], max_results=mr, limit=lim, user_auth=True
                ).flatten()):
    insert_ind = 0
    for ind, sorted_user in enumerate(sorted_flwrs):
        if user.public_metrics['followers_count'] >= sorted_user.public_metrics['followers_count']:
            insert_ind = ind
            break
    sorted_flwrs.insert(insert_ind, user)

In [ ]:
for i, user in enumerate(sorted_flwrs, 1):
    flwrs_cnt = user.public_metrics['followers_count']
    bold = flwrs_cnt > 100000
    display(HTML(f'''
        {i}<br>
        {'<b style="color:red;">' if bold else ''}
        <a href='https://twitter.com/{user.username}'>
            <img src={user.profile_image_url}><br>
            {user.name} (@{user.username})
        </a>
        <div>
            {flwrs_cnt} followers<br>
            {user.public_metrics['following_count']} followings
        </div>
        {'</b>' if bold else ''}
    '''))

#### Tweets insight using `.get_users_tweets()`

1 <= `max_results` <= 100

In [ ]:
mr = 100
lim = target['public_metrics']['tweet_count'] // mr + 1
# lim = 1
tweet_list = []
reply_list = []
for tweet in tqdm(tweepy.Paginator(client.get_users_tweets,
                    target['id'], tweet_fields=['in_reply_to_user_id'], max_results=mr, limit=lim, user_auth=True
                ).flatten()):
    reply_to = tweet.in_reply_to_user_id
    if reply_to is None:
        tweet_list.append(tweet)
        continue
    if reply_to == target['id']:
        tweet_list.append(tweet)
        continue
    reply_list.append(tweet)
    print(tweet.text)
    print("---")

In [ ]:
reply_list.__len__()

In [ ]:
user_dict = {}
for tweet in tqdm(reply_list):
    text = tweet.text
    text = text.split()
    text = ' '.join([word for word in text if not word.startswith('@')])
    # remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    words = text.split()
    for word in words:
        if word in user_dict:
            user_dict[word] += 1
        else:
            user_dict[word] = 1

In [ ]:
# sort dictionary by value
sorted_dictionary = sorted(user_dict.items(), key=lambda kv: kv[1], reverse=True)
sorted_dictionary